In [3]:
def inputify(fileName):
    return fileName
def outputify(fileName):
    return fileName

In [4]:
%matplotlib widget

import pandas as pd
import json
import numpy as np
import csv
import os
import matplotlib
import matplotlib.pyplot as plt
import pickle
from xgboost import XGBClassifier
from xgboost import plot_importance
from matplotlib import pyplot
from sklearn.externals import joblib
from sklearn.mixture import GaussianMixture
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import KMeans
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from time import localtime
from datetime import datetime as dt
from matplotlib import pyplot as plt
from scipy.spatial.distance import cdist
from pathlib import Path

working_directory     = str(os.getcwd()) + "/"
# working_directoryPT = str(Path.home()) + "/"
# wd                  = "/Users/Jonat/Dropbox/CreateDawg/ImRight/input/"

In [25]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix 

def hyperTune(model,X_Train,Y_Train,X_Test,Y_Test):
    #Hyperparameter find utilizing grid search cross validation 
    C = [1, 10, 100, 1000]
    mItr = [500]
    tol = [.001,.01]
#     class_weight = [{'Blue':0.5, 'Red':0.5}, {'Blue':0.4, 'Red':0.6}, {'Blue':0.6, 'Red':0.4}, {'Blue':0.7, 'Red':0.3},'none']
    solver = ['newton-cg', 'lbfgs','sag','saga']
    param_grid = dict(tol=tol,max_iter=mItr, C=C,  solver=solver)
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    grid = GridSearchCV(estimator=model,param_grid=param_grid,cv=cv,scoring='roc_auc',verbose=1,n_jobs=-1)
    grid_result = grid.fit(X_Train, Y_Train)
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

#     print("========================================================")
#     print('Train Score: ', grid_result.best_score_)
#     print('Best Train Params: ', grid_result.best_params_)
#     # print('Validation score:',grid.score(X_Val,Y_Val))
#     print('Test score:',grid.score(X_Test,Y_Test))
# #     return grid_result.best_params_
    return grid_result.best_params_ , grid.score(X_Test, Y_Test)
def hyperTuneXGB(model,X_Train,Y_Train,X_Test,Y_Test):
    #Hyperparameter find utilizing grid search cross validation 
    params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'alpha': [0.0001, 0.05,0.1],
        'max_depth': [3, 4, 5]
    }
#     cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    grid = GridSearchCV(estimator=model,param_grid=params,cv=5,scoring='roc_auc',verbose=1,n_jobs=-1)
    grid_result = grid.fit(X_Train, Y_Train)
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))
    return grid_result.best_params_ , grid.score(X_Test, Y_Test)
def hyperTuneSVC(model,X_Train,Y_Train,X_Test,Y_Test):
    #Hyperparameter find utilizing grid search cross validation 
    params = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['linear','poly','rbf','sigmoid']}  
#     cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    grid = GridSearchCV(estimator=model,param_grid=params,cv=5,scoring='roc_auc',verbose=1,n_jobs=-1)
    grid_result = grid.fit(X_Train, Y_Train)
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))
    return grid_result.best_params_ , grid.score(X_Test, Y_Test)
def hyperTuneRF(model,X_Train,Y_Train,X_Test,Y_Test):
    # Number of trees in random forest
    n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
    # Number of features to consider at every split
    max_features = ['auto', 'sqrt']
    # Maximum number of levels in tree
    max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
    max_depth.append(None)
    # Minimum number of samples required to split a node
    min_samples_split = [2, 5, 10]
    # Minimum number of samples required at each leaf node
    min_samples_leaf = [1, 2, 4]
    # Method of selecting samples for training each tree
    bootstrap = [True, False]
    # Create the random grid
    params    = {'n_estimators': n_estimators,'max_features': max_features,'max_depth': max_depth,'min_samples_split': min_samples_split,'min_samples_leaf': min_samples_leaf,'bootstrap': bootstrap}
    grid = GridSearchCV(estimator=model,param_grid=params,cv=5,scoring='roc_auc',verbose=1,n_jobs=-1)
    grid_result = grid.fit(X_Train, Y_Train)
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))
    return grid_result.best_params_ , grid.score(X_Test, Y_Test)
def hyperTuneMLP(model,X_Train,Y_Train,X_Test,Y_Test):
    #Hyperparameter find utilizing grid search cross validation 
    parameter_space = {
    'hidden_layer_sizes': [(72,),(107,),(107,214,107),(107,50,20)],
    'activation': ['tanh', 'relu','logistic'],
    'solver': ['sgd', 'adam','lbfgs'],
    'alpha': [0.0001, 0.05,0.1],
    'learning_rate': ['constant','adaptive','invscaling'],
    }
#     cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    grid = GridSearchCV(estimator=model,param_grid=parameter_space,cv=5,scoring='roc_auc',verbose=1,n_jobs=-1)
    grid_result = grid.fit(X_Train, Y_Train)
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

#     print("========================================================")
#     print('Train Score: ', grid_result.best_score_)
#     print('Best Train Params: ', grid_result.best_params_)
#     # print('Validation score:',grid.score(X_Val,Y_Val))
#     print('Test score:',grid.score(X_Test,Y_Test))
# #     return grid_result.best_params_
    return grid_result.best_params_ , grid.score(X_Test, Y_Test)
def gnb(X_Train,Y_Train,X_Test,Y_Test):
    from sklearn.naive_bayes import GaussianNB
    from sklearn.metrics import accuracy_score
    clf    = GaussianNB()
    clf.fit(X_Train,Y_Train)
    y_pred = clf.predict(X_Test)
    acc    = accuracy_score(Y_Test,y_pred)
    return clf, acc
def mlp(X_Train,Y_Train,X_Test,Y_Test):
    from sklearn.neural_network import MLPClassifier
    from sklearn.metrics import accuracy_score
#     {'activation': 'tanh', 'alpha': 0.05, 'hidden_layer_sizes': (72,), 'learning_rate': 'invscaling', 'solver': 'adam'}
#   {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (107, 214, 107), 'learning_rate': 'adaptive', 'solver': 'adam'}
    clf = MLPClassifier(max_iter=10000,activation= 'tanh', alpha= 0.0001,tol=.001, hidden_layer_sizes= (107, 214, 107), learning_rate= 'adaptive', solver= 'adam')
    clf.fit(X_Train,Y_Train)
    y_pred = clf.predict(X_Test)
    acc    = accuracy_score(Y_Test,y_pred)
    return clf, acc
#     clf = MLPClassifier(max_iter=1)

#     hyperparams, bestScore = hyperTuneMLP(clf,X_Train,Y_Train,X_Test,Y_Test)
#     return hyperparams, bestScore
    
def rforest(X_Train,Y_Train,X_Test,Y_Test):
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import accuracy_score
#     clf = RandomForestClassifier()
#     clf.fit(X_Train,Y_Train)
#     y_pred = clf.predict(X_Test)
#     acc    = accuracy_score(Y_Test,y_pred)
#     #     ,classification_report(Y_Test, predictions)
#     return clf, acc

    clf = RandomForestClassifier()
    hyperparams, bestScore = hyperTuneRF(clf,X_Train,Y_Train,X_Test,Y_Test)
    return hyperparams, bestScore

def boosted(X_Train,Y_Train,X_Test,Y_Test):
    from xgboost import XGBClassifier
    from sklearn.metrics import accuracy_score
#     {'alpha': 0.05, 'colsample_bytree': 1.0, 'gamma': 5, 'max_depth': 3, 'min_child_weight': 10, 'subsample': 1.0}
    boost    = XGBClassifier(booster='gbtree',alpha='.05',colsample_bytree =1.0, gamma = 5, max_depth = 3, min_child_weight = 10, subsample= 1.0)
    boost.fit(X_Train,Y_Train)
    y_pred   = boost.predict(X_Test)
    accuracy = accuracy_score(Y_Test,y_pred)
#     ,classification_report(Y_Test, predictions)
    return boost, accuracy

#     boost = XGBClassifier()
#     hyperparams, bestScore = hyperTuneXGB(boost,X_Train,Y_Train,X_Test,Y_Test)
#     return hyperparams, bestScore
def boosted2(X_Train,Y_Train,X_Test,Y_Test):
    from xgboost import XGBClassifier
    from sklearn.metrics import accuracy_score
#     {'alpha': 0.05, 'colsample_bytree': 1.0, 'gamma': 5, 'max_depth': 3, 'min_child_weight': 10, 'subsample': 1.0}
    boost    = XGBClassifier(alpha = 0.1, colsample_bytree = 1.0, gamma = 5, max_depth = 3, min_child_weight = 10, subsample = 1.0)
    boost.fit(X_Train,Y_Train)
    y_pred   = boost.predict(X_Test)
    accuracy = accuracy_score(Y_Test,y_pred)
#     ,classification_report(Y_Test, predictions)
    return boost, accuracy,classification_report(Y_Test, y_pred)

#     boost = XGBClassifier()
#     hyperparams, bestScore = hyperTuneXGB(boost,X_Train,Y_Train,X_Test,Y_Test)
#     return hyperparams, bestScore
def svc(X_Train,Y_Train,X_Test,Y_Test):
    from sklearn.svm import SVC
    from sklearn.metrics import accuracy_score
#     {'alpha': 0.05, 'colsample_bytree': 1.0, 'gamma': 5, 'max_depth': 3, 'min_child_weight': 10, 'subsample': 1.0}
#     C = 100, gamma = 0.0001, kernel = 'rbf'
    svc    = SVC(C = 1, gamma = 0.1, kernel = 'sigmoid',probability=True)
    svc.fit(X_Train,Y_Train)
    y_pred   = svc.predict(X_Test)
    accuracy = accuracy_score(Y_Test,y_pred)
#     ,classification_report(Y_Test, y_pred)
    return svc, accuracy, classification_report(Y_Test, y_pred)
#     svc = SVC()
#     hyperparams, bestScore = hyperTuneSVC(svc,X_Train,Y_Train,X_Test,Y_Test)
#     return hyperparams, bestScore
def logReg(X_Train,Y_Train,X_Test,Y_Test):
#     {'C': 1000, 'max_iter': 500, 'solver': 'newton-cg', 'tol': 0.001}
#    {'C': 1000, 'max_iter': 500, 'solver': 'newton-cg', 'tol': 0.001}
#     logRegg = LogisticRegression(class_weight = 'none', max_iter= 10000, tol = 0.001, solver= 'newton-cg')
#     logRegg.fit(X_Train,Y_Train)
#     # model   = SelectFromModel(logRegg,prefit=True)
#     # x_new   = model.transform(X_Train)
#     # chopp   = model.get_support()
#     # for i in range(len(Headers)):
#     # if(chopp[i]==True):
#     # # print(Headers[i])
#     print("G: "+g+" Score: ",logRegg.score(X_Test,Y_Test))
#     return logRegg, logRegg.score(X_Test,Y_Test)
#     print("================================================================")

    # feature selection

#     logRegg = LogisticRegression()
#     # get the hyperparameters
#     try:
#         hyperparams, bestScore = hyperTune(logRegg,X_Train,Y_Train,X_Test,Y_Test)
#         return hyperparams, bestScore
#     except ValueError:
#         print('lolwut')
    logRegg = LogisticRegression(C = 10, max_iter = 500, solver = 'sag', tol = 0.01,random_state = 42)
    logRegg.fit(X_Train,Y_Train)
    predictions = logRegg.predict(X_Test) 
#     ,classification_report(Y_Test, predictions)
    return logRegg, logRegg.score(X_Test,Y_Test),classification_report(Y_Test, predictions)

In [26]:
fPM        = pd.read_csv('LCS_Data/playerMatchUps.csv')
fOD        = pd.read_csv('LCS_Data/objectiveData.csv')
gameAmount = len(list(set(list(fPM['gamenum']))))
print(fPM.shape, fOD.shape)

(13095, 47) (2619, 36)


In [27]:
# splitting data for stacks
fPM_1 = fPM[fPM['gamenum']<(gameAmount/2)].copy()
fOD_1 = fOD[fOD['gamenum']<(gameAmount/2)].copy()
fPM_2 = fPM[fPM['gamenum']>(gameAmount/2)].copy()
fOD_2 = fOD[fOD['gamenum']>(gameAmount/2)].copy()

In [54]:
# ========================================== first layer ======================================== #

# ***************************  pvp model  ******************************
print('PVP Results:')
positions = ['top','jng','mid','bot','sup']
for pos in positions:
    relevantD   = fPM_1[fPM_1['position'] == pos].copy() 
    relevantD.drop(['gamenum','position'],axis=1,inplace=True)
    # pvp matchup model creation
    train, test = train_test_split(relevantD.copy(),test_size=.1,random_state=33)
    # test, val   = train_test_split(test.copy(),test_size=.5)
    y_train     = train['res']
    y_test      = test['res']
    # y_val       = val['res']
    train.drop(['res'],axis=1,inplace=True)
    test.drop(['res'],axis=1,inplace=True)
    # val.drop(['res'],axis=1,inplace=True)

    scalersPVP = Normalizer()
    train   = scalersPVP.fit_transform(train)
    test    = scalersPVP.transform(test)
    
    modelPVP,scorePVP = boosted(train,y_train,test,y_test)
    print(pos,scorePVP)
    joblib.dump(modelPVP,outputify(working_directory+"pvpModels_LCS/"+pos+"_2.pkl"))
    joblib.dump(scalersPVP,outputify(working_directory+"scalers_LCS/"+pos+"_scaler_2.pkl"))


    
print()
# *****************************  objective model **************************
relevantD_OD   = fOD_1.copy() 
relevantD_OD.drop(['gamenum'],axis=1,inplace=True)
# pvp matchup model creation
train, test = train_test_split(relevantD_OD.copy(),test_size=.1,random_state=34)
# test, val   = train_test_split(test.copy(),test_size=.5)
y_train     = train['res']
y_test      = test['res']
# y_val       = val['res']
train.drop(['res'],axis=1,inplace=True)
test.drop(['res'],axis=1,inplace=True)
# val.drop(['res'],axis=1,inplace=True)

scalersO = Normalizer()
train   = scalersO.fit_transform(train)
test    = scalersO.transform(test)

modelO,scoreO = logReg(train,y_train,test,y_test)[:2]
print("Objective Acc:",scoreO)


joblib.dump(modelO,outputify(working_directory+"objectiveModels_LCS/obj_2.pkl"))
joblib.dump(scalersO,outputify(working_directory+"scalers_LCS/sObj_2.pkl"))
# =================================== end of first layer ======================================== #


PVP Results:
top 0.549618320610687
jng 0.5648854961832062
mid 0.6106870229007634
bot 0.5648854961832062
sup 0.6183206106870229

Objective Acc: 0.5954198473282443


['/mnt/c/Users/Jonat/Dropbox/CreateDawg/3SC/scalers_LCS/sObj_2.pkl']

In [55]:
# ================================================ Creating The Stacked Dataset ========================================
stackedTbl  = []
results     = list(fOD_2['res'])
gameNumbers = sorted(list(set(list(fPM_2['gamenum']))))

modelObj    = joblib.load("objectiveModels_LCS/obj_2.pkl")
scalerObj   = joblib.load("scalers_LCS/sObj_2.pkl")
for game in gameNumbers:
    teamR = []
    teamB = []
    gameDataPVP = fPM_2[fPM_2['gamenum']==game].copy()
    gameDataObj = fOD_2[fOD_2['gamenum']==game].copy()
    for pos in positions:
        relevantD = gameDataPVP[gameDataPVP['position']==pos].copy()
        relevantD.drop(['res','position','gamenum'],inplace=True,axis=1)
        
        relevantM = joblib.load("pvpModels_LCS/"+pos+"_2.pkl")
        relevantS = joblib.load("scalers_LCS/"+pos+"_scaler_2.pkl")
        
        scaledRD  = relevantS.transform(relevantD)
        pvpProbs  = relevantM.predict_proba(scaledRD)
        teamR.append(pvpProbs[0][0])
        teamB.append(pvpProbs[0][1])
    gameDataObj.drop(['res','gamenum'],inplace=True,axis=1)
    scaledObjData = scalerObj.transform(gameDataObj)
#     objResult = modelObj.predict(scaledObjData)
#     teamR.append(objResult[0])
    teamB.append(results[game-(len(gameNumbers)+2)])
    stackedTbl.append(teamR + teamB)

In [56]:
# ================================================ Turning Dataset to DataFrame ===============================
rl = []
bl = []
for pos in positions:
    rl.append("r_"+pos+'_pred')
    bl.append("b_"+pos+'_pred')


stackedDF = pd.DataFrame(data=stackedTbl, columns = rl + bl + ['res'])

In [57]:
with open("LCS_Data/moesStack_3.csv","w",newline='\n') as f:
    stackedDF.to_csv(f,index=False,sep=',',header=True)
stackedDF

,r_top_pred,r_jng_pred,r_mid_pred,r_bot_pred,r_sup_pred,b_top_pred,b_jng_pred,b_mid_pred,b_bot_pred,b_sup_pred,res
0,0.258924,0.504373,0.468965,0.263403,0.368222,0.741076,0.495627,0.531035,0.736597,0.631778,Red
1,0.472266,0.553438,0.348270,0.509939,0.485140,0.527734,0.446562,0.651730,0.490061,0.514860,Red
2,0.486890,0.473362,0.409792,0.622042,0.572118,0.513110,0.526638,0.590208,0.377958,0.427882,Blue
3,0.560639,0.470540,0.607533,0.376701,0.355752,0.439361,0.529460,0.392467,0.623299,0.644248,Red
4,0.446450,0.492461,0.495152,0.658742,0.742684,0.553550,0.507539,0.504848,0.341258,0.257316,Red
...,...,...,...,...,...,...,...,...,...,...,...
1304,0.475795,0.552443,0.399313,0.643444,0.606500,0.524205,0.447557,0.600687,0.356556,0.393500,Blue
1305,0.569391,0.643768,0.673916,0.622685,0.695256,0.430609,0.356232,0.326084,0.377315,0.304744,Blue
1306,0.506864,0.502010,0.432814,0.219057,0.315329,0.493136,0.497990,0.567186,0.780943,0.684671,Blue
1307,0.581299,0.483050,0.788847,0.807479,0.471287,0.418701,0.516950,0.211153,0.192521,0.528713,Blue


In [58]:
# ========================================== bottom layer ======================================== #
best = 0
bScore = -999
scores = []
for i in range(1000):
    tempData = stackedDF.copy()
#     tempData.obj_pred.replace(('Red', 'Blue'), (0, 1), inplace=True)
    train, test = train_test_split(tempData.copy(),test_size=.1,random_state=i)
    # test, val   = train_test_split(test.copy(),test_size=.5)
    y_train     = train['res']
    y_test      = test['res']
    # y_val       = val['res']
    train.drop(['res'],axis=1,inplace=True)
    test.drop(['res'],axis=1,inplace=True)
    # val.drop(['res'],axis=1,inplace=True)

    scalers = Normalizer()
    train   = scalers.fit_transform(train)
    test    = scalers.transform(test)

    model,score = boosted2(train,y_train,test,y_test)[:2]
    scores.append(score)
    if(score>bScore):
        bScore=score
        best=i


/home/koppej/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [59]:
tempData = stackedDF.copy()
# tempData.obj_pred.replace(('Red', 'Blue'), (0, 1), inplace=True)
train, test = train_test_split(tempData.copy(),test_size=.1,random_state=best)
# test, val   = train_test_split(test.copy(),test_size=.5)
y_train     = train['res']
y_test      = test['res']
# y_val       = val['res']
train.drop(['res'],axis=1,inplace=True)
test.drop(['res'],axis=1,inplace=True)
# val.drop(['res'],axis=1,inplace=True)

scalers = Normalizer()
train   = scalers.fit_transform(train)
test    = scalers.transform(test)

# model,score = logReg(train,y_train,test,y_test)
# print(score)
model = boosted2(train,y_train,test,y_test)[0]
print(boosted2(train,y_train,test,y_test)[2])

              precision    recall  f1-score   support

        Blue       0.60      0.51      0.55        57
         Red       0.66      0.74      0.70        74

    accuracy                           0.64       131
   macro avg       0.63      0.63      0.63       131
weighted avg       0.64      0.64      0.64       131



In [60]:
len(y_test)


131

In [61]:
nlist = sorted(scores)
nlist[(len(nlist)-1)//2]


0.5190839694656488

In [62]:
sum(nlist)/len(nlist)

0.5185419847328226

In [63]:
model

XGBClassifier(alpha=0.1, base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1.0, gamma=5, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.300000012, max_delta_step=0, max_depth=3,
              min_child_weight=10, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0.100000001, reg_lambda=1, scale_pos_weight=1,
              subsample=1.0, tree_method=None, validate_parameters=False,
              verbosity=None)

In [64]:
joblib.dump(model,outputify(working_directory+"finalModel_LCS/final_3.pkl"))
joblib.dump(scalers,outputify(working_directory+"scalers_LCS/final_3.pkl"))


['/mnt/c/Users/Jonat/Dropbox/CreateDawg/3SC/scalers_LCS/final_3.pkl']

In [ ]:
stackedTbl  = []
results     = list(fOD_2['res'])
gameNumbers = sorted(list(set(list(fPM_2['gamenum']))))

modelObj    = joblib.load("objectiveModels_LCK/obj_2.pkl")
scalerObj   = joblib.load("scalers_LCK/sObj_2.pkl")
for game in gameNumbers:
    teamR = []
    teamB = []
    gameDataPVP = fPM_2[fPM_2['gamenum']==game].copy()
    gameDataObj = fOD_2[fOD_2['gamenum']==game].copy()
    for pos in positions:
        relevantD = gameDataPVP[gameDataPVP['position']==pos].copy()
        relevantD.drop(['res','position','gamenum'],inplace=True,axis=1)
        
        relevantM = joblib.load("pvpModels_LCK/"+pos+"_2.pkl")
        relevantS = joblib.load("scalers_LCK/"+pos+"_scaler_2.pkl")
        
        scaledRD  = relevantS.transform(relevantD)
        pvpProbs  = relevantM.predict_proba(scaledRD)
        teamR.append(pvpProbs[0][0])
        teamB.append(pvpProbs[0][1])
    gameDataObj.drop(['res','gamenum'],inplace=True,axis=1)
    scaledObjData = scalerObj.transform(gameDataObj)
    objResult = modelObj.predict(scaledObjData)
    teamR.append(objResult[0])
    teamB.append(results[game-(len(gameNumbers)+2)])
    stackedTbl.append(teamR + teamB)